In [89]:
from data4ml import Data4TextGeneration
from time import time
from collections import defaultdict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
data4gen = Data4TextGeneration(path_to_config='./../data_params.json')
data4gen.home_folder = './../../messages'
res = data4gen.make_data(limit=2)

lol


In [30]:
data4gen.clear_message(data4gen.parse_html('./..', ['messages0.html']))[5:9]

['Дружочек',
 'Во сколько за деньгами поедешь ?)',
 'Хаюшки',
 'Я в понедельник поеду']

In [111]:
import re

def make_first_char_upper(sent: list):
    sent[0] = sent[0][0].upper() + sent[0][1:]
    
def add_dot_in_the_end(sent: list):
    sent[-1] = sent[-1] + '.' if sent[-1][-1].isalpha() else sent[-1]
    
def tokenize(corpus: list) -> list:
    tokenize_messages = []
    for dialog in corpus:
        for message in dialog:
            sentence = [re.sub("[.,:@-]", "", i) for i in message.lower().split() if len(re.sub("[.,:@-]", "", i)) > 0]
            if sentence:
                make_first_char_upper(sentence)
                add_dot_in_the_end(sentence)
                tokenize_messages.extend(sentence)
    return tokenize_messages

In [112]:
tok_res = tokenize(res)

In [115]:
res[0][:30]

['Когда дома будешь?',
 'К вечеру ближе',
 'Окей',
 '15:00 - 19:00 ТОРГЛАЙН ООО Чкалова ул., 28А Анастасия Петриченко',
 'На завтра',
 'Будь в за минут 15 заранее',
 'И напиши во сколько ты сегодня будешь дома',
 'Чтобы я тебе всё передал',
 'Окей',
 'Ты завтра работать не планируешь что ли?',
 'Мы же должны были сегодня встретиться',
 'Как не планирую ?',
 'Ааааа, не заметила 😑😑😑',
 'Можем утром встретиться ?',
 'Нууу',
 'Давай утром',
 'Или днём',
 'Напиши где и во сколько ко',
 'Хз как пойдёт',
 'Давай завтра уже',
 'Хорошо',
 'Добрый день )',
 'В футболке чёрной можно ?',
 'Спасибо',
 'А с тобой как ?',
 'Тогда с тобой',
 'Окей',
 'Ты знаешь, где магазин Альфа?',
 'Какой из 3-х?',
 'На Дергаевской']

In [117]:
print(tok_res[:60])

['Когда', 'дома', 'будешь?', 'К', 'вечеру', 'ближе.', 'Окей.', '1500', '1900', 'торглайн', 'ооо', 'чкалова', 'ул', '28а', 'анастасия', 'петриченко.', 'На', 'завтра.', 'Будь', 'в', 'за', 'минут', '15', 'заранее.', 'И', 'напиши', 'во', 'сколько', 'ты', 'сегодня', 'будешь', 'дома.', 'Чтобы', 'я', 'тебе', 'всё', 'передал.', 'Окей.', 'Ты', 'завтра', 'работать', 'не', 'планируешь', 'что', 'ли?', 'Мы', 'же', 'должны', 'были', 'сегодня', 'встретиться.', 'Как', 'не', 'планирую', '?', 'Ааааа', 'не', 'заметила', '😑😑😑', 'Можем']


In [3]:
len(res)

96

In [83]:
RES = []

for dialogs in res:
    ss = []
    for message in dialogs:
        ss.extend([x.lower().replace('[,.?]', '') for x in message.split()])
    RES.append(ss)

In [81]:
len(RES)

96

In [84]:
print(RES[1][:10])

['серега,', 'привету', 'тебя', 'есть', 'телевизор', 'на', 'даче?', 'просто', 'если', 'что,']


In [65]:
sentences = bigram[sent]

NameError: name 'bigram' is not defined

In [60]:
from gensim.models.phrases import Phrases, Phraser

In [58]:
phrases = Phrases(RES, min_count=30, progress_per=10000)

In [66]:
bigram = Phraser(phrases)

In [69]:
sentences = bigram[RES]

In [72]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

321400

In [73]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['не', 'Я', 'я', 'и', 'в', 'А', 'Ну', 'И', 'ты', 'на']

In [74]:
w2v_mode = kek(sentences)

Time to build vocab: 0.17 mins
Time to train the model: 3.78 mins


In [12]:
import multiprocessing

from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

In [13]:
cores = multiprocessing.cpu_count()
cores

4

In [34]:
def kek(res):
    w2v_model = Word2Vec(min_count=20,
                         window=2,
                         size=300,
                         sample=6e-5, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         negative=20,
                         workers=cores-1)

    t = time()

    w2v_model.build_vocab(res, progress_per=10000)

    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

    w2v_model.corpus_count

    t = time()

    w2v_model.train(res, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

    w2v_model.init_sims(replace=True)

    return w2v_model

In [35]:
w2v_model_1 = kek(RES)

Time to build vocab: 0.05 mins
Time to train the model: 0.44 mins


In [78]:
w2v_mode.wv.most_similar(positive=["Юля"])

[('Написала', 0.8950090408325195),
 ('Ване', 0.8853252530097961),
 ('говорит?', 0.8794123530387878),
 ('ВСЕ', 0.8751375079154968),
 ('звонила?', 0.8749449253082275),
 ('Богу', 0.8741962909698486),
 ('покажи', 0.8736038208007812),
 ('надоел', 0.8734161853790283),
 ('Ахахахаах', 0.8711090087890625),
 ('вк?', 0.8618061542510986)]

In [79]:
w2v_model_1.wv.most_similar(positive=["Юля"])

[('память', 0.7860834002494812),
 ('инсту', 0.7789162397384644),
 ('Ии', 0.7788558006286621),
 ('недоступна', 0.7754437923431396),
 ('покажи', 0.7736333608627319),
 ('берешь', 0.7618379592895508),
 ('Ане', 0.7555817365646362),
 ('Эту', 0.7530922293663025),
 ('фотки)', 0.7501952648162842),
 ('Ване', 0.7494977116584778)]